In [63]:
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt 
import numpy as np 
import os 
import pandas as pd 
import keras
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt
from matplotlib.image import imread
from PIL import Image

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.preprocessing import OneHotEncoder

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
import tensorflow as tf
import re

from sklearn.model_selection import KFold


#print how many rows and columns are there in the csv file
df = pd.read_csv('train-balanced-sarcasm.csv')


df_useful_x = df[['comment','label']]
df_useful_y = df[['label']]
df_useful_x['comment'] = df_useful_x['comment'].astype('str')

df_useful_x.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1010826 entries, 0 to 1010825
Data columns (total 2 columns):
comment    1010826 non-null object
label      1010826 non-null int64
dtypes: int64(1), object(1)
memory usage: 15.4+ MB


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [65]:
df_useful_x

comment  label  \
0                                               NC and NH.      0   
1        You do know west teams play against west teams...      0   
2        They were underdogs earlier today, but since G...      0   
3        This meme isn't funny none of the "new york ni...      0   
4                          I could use one of those tools.      0   
...                                                    ...    ...   
1010821  I'm sure that Iran and N. Korea have the techn...      1   
1010822                 whatever you do, don't vote green!      1   
1010823  Perhaps this is an atheist conspiracy to make ...      1   
1010824  The Slavs got their own country - it is called...      1   
1010825  values, as in capitalism .. there is good mone...      1   

         pre_clean_len  
0                   10  
1                   74  
2                  121  
3                   60  
4                   31  
...                ...  
1010821             92  
1010822             34  
1010823             66  
1010824             53  
1010825             72  

[1010826 rows x 3 columns]

In [66]:
df_useful_x['pre_clean_len'] = [len(t) for t in df_useful_x.comment]
df_useful_x

df_useful_x[df_useful_x.pre_clean_len > 140].head(10)
df_useful_x.count

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


<bound method DataFrame.count of                                                    comment  label  \
0                                               NC and NH.      0   
1        You do know west teams play against west teams...      0   
2        They were underdogs earlier today, but since G...      0   
3        This meme isn't funny none of the "new york ni...      0   
4                          I could use one of those tools.      0   
...                                                    ...    ...   
1010821  I'm sure that Iran and N. Korea have the techn...      1   
1010822                 whatever you do, don't vote green!      1   
1010823  Perhaps this is an atheist conspiracy to make ...      1   
1010824  The Slavs got their own country - it is called...      1   
1010825  values, as in capitalism .. there is good mone...      1   

         pre_clean_len  
0                   10  
1                   74  
2                  121  
3                   60  
4            

In [68]:
from bs4 import BeautifulSoup
from nltk.tokenize import WordPunctTokenizer
tok = WordPunctTokenizer()
pat1 = r'@[A-Za-z0-9]+'
pat2 = r'https?://[A-Za-z0-9./]+'
combined_pat = r'|'.join((pat1, pat2))

def text_cleaner(text):
    soup = BeautifulSoup(text, 'lxml')
    souped = soup.get_text()
    stripped = re.sub(combined_pat, '', souped)
    try:
        clean = stripped.decode("utf-8-sig").replace(u"\ufffd", "?")
    except:
        clean = stripped
    letters_only = re.sub("[^a-zA-Z]", " ", clean)
    lower_case = letters_only.lower()
    # During the letters_only process two lines above, it has created unnecessay white spaces,
    # I will tokenize and join together to remove unneccessary white spaces
    words = tok.tokenize(lower_case)
    return (" ".join(words)).strip()



nums = [0,1010826]
print ("Cleaning and parsing the comments...\n")
clean_comment_texts = []
for i in range(nums[0],nums[1]):
    if( (i+1)%10000 == 0 ):
        print ("comments %d of %d has been processed" % ( i+1, nums[1] ))                                                                    
    clean_comment_texts.append(text_cleaner(df_useful_x['comment'][i]))
    
clean_df = pd.DataFrame(clean_comment_texts,columns=['comment'])
clean_df['sentiment'] = df_useful_x.label
clean_df.head()


Cleaning and parsing the comments...



KeyboardInterrupt: 

In [25]:
#show the columns to see which ones we would need to use, and make a subdataframe with columns we need

# convert each dataframe into 2D numpy arrays
x= df_useful_x.to_numpy()
y= df_useful_y.to_numpy()

Index(['label', 'comment', 'author', 'subreddit', 'score', 'ups', 'downs',
       'date', 'created_utc', 'parent_comment'],
      dtype='object')


In [24]:
x


array([['NC and NH.'],
       ['You do know west teams play against west teams more than east teams right?'],
       ["They were underdogs earlier today, but since Gronk's announcement this afternoon, the Vegas line has moved to patriots -1"],
       ...,
       ['Perhaps this is an atheist conspiracy to make Christians look bad?'],
       ['The Slavs got their own country - it is called Kosovo'],
       ['values, as in capitalism .. there is good money in imprisoning people ..']],
      dtype=object)

In [26]:
y

array([[0],
       [0],
       [0],
       ...,
       [1],
       [1],
       [1]])

In [36]:
from sklearn.model_selection import train_test_split

X_train,X_test, Y_train, Y_test = train_test_split(x,y,test_size=0.2,random_state=0)

In [37]:
X_train

array([['Looks more deformed than half elephant..'],
       ["If you want our president to be killed you are not a patriot, you're a fucking traitor."],
       ['"Acceptance"'],
       ...,
       ['agreed , This vid shows how much better this guy is then kennyS at his best'],
       ['Hang ingat hang mat salleh ka nyanyi lagu ongputeh?'],
       ["From what I've read, inferno isn't as hard as bliz made it out to be, they just have not-hardcore testers."]],
      dtype=object)

In [38]:
X_test

array([['Das racist'],
       ['Ugh, people are always complaining about us 1%ers.'],
       ['But using the lightning port means that there will have to be a DAC in the adapter (not sure about an amp), and it will probably be low quality.'],
       ...,
       ['Da buon padre fa la cosa giusta e avvisalo, o quantomeno insegnagli a cucinare'],
       ["Why don't you let the parent do the parenting?"],
       ['I really hope you forgot to put']], dtype=object)

In [39]:
Y_train

array([[1],
       [0],
       [0],
       ...,
       [1],
       [1],
       [0]])

In [40]:
Y_test

array([[0],
       [1],
       [0],
       ...,
       [1],
       [0],
       [1]])

In [41]:
print('Reddit Dataset Shape:')
print('X_train: ' + str(X_train.shape))
print('Y_train: ' + str(Y_train.shape))
print('X_test:  '  + str(X_test.shape))
print('Y_test:  '  + str(Y_test.shape))

Reddit Dataset Shape:
X_train: (808660, 1)
Y_train: (808660, 1)
X_test:  (202166, 1)
Y_test:  (202166, 1)


In [42]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
max_words=500
X_train = X_train[1:200]
Y_train = Y_train[1:200]

X_test = X_test[1:200]
Y_test = Y_test[1:200]

# pad upto max_words
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

ValueError: invalid literal for int() with base 10: "If you want our president to be killed you are not a patriot, you're a fucking traitor."